# This notebook is focused on the questions asked in each survey. Here the questions are grouped by related they are to each other and further analysis is performed. 

## Authorization and accessing the data

In [1]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 16.4MB/s 
     |████████████████████████████████| 3.0MB 29.8MB/s 
     |████████████████████████████████| 1.2MB 32.4MB/s 
     |████████████████████████████████| 870kB 35.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=94e77a019ed4b541bd327fd0ef427bec5eb7d71333ce83c5c617ff5609920c62
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [14]:
#libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import pipeline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

##Stored Questions for K-survey

In [15]:
q_cols = ['general empl satisfaction',
 'recommend empl unit to friend',
 'motivation of direct colleagues you are working with day to day? *Top 3',
 'My direct colleagues are coping well with the change and transformation in our division/function',
 'People in my team frequently go above and beyond the requirements of the job',
 'I am proud working for my company and gladly tell people about it',
 'I believe strongly in and support the future direction of my company',
 'I trust that my company takes action that balances the best interests of our people, business and clients',
 "My company's handling of this year’s challenges leaves me confident in our future success",
 "My company's future direction leaves me confident about my career opportunities here",
 'In my team we continuously use client feedback to improve our products/services',
 'In my team we have an ongoing dialogue about our clients and their requirements and expectations',
 'Compared to last year, our clients now view us as:',
 'My immediate manager creates an atmosphere of openness and trust',
 'I have confidence in my immediate manager',
 'I have confidence in the global senior management of [division]',
 'I have confidence in the senior management of [FUNCTION: Finance, HR, Technology, Operations, Marketing]',
 'I have confidence in my local/country senior management',
 'As a member of [FUNCTION: Finance, HR, Technology, Operations, Marketing] I am familiar with the overall objectives & strategies of my function',
 'I am familiar with the overall objectives & strategies of [division]',
 'I can clearly see how my own work contributes to the overall objectives & strategies of [division]',
 'In my team we work towards clear objectives',
 'In my team we make decisions rapidly when it is necessary',
 'In my team actions are taken quickly when decisions have been made',
 'Please rate the cooperation between different units within [division]',
 'Please rate the cooperation within [FUNCTION: Finance, HR, Technology, Operations, Marketing]',
 'Please rate the cooperation across the company as a whole',
 'In my team new ideas receive very strong support and encouragement',
 'I feel I can make my own decisions concerning my work',
 "I have seen action taken based on the results of last year's survey",
 'In my company diversity of skills, experiences, background and ways of working are recognized and appreciated',
 'My immediate manager treats all team members fairly, regardless of age, gender identity, sex, family status, race, national origin, nationality, religion, disability or sexual orientation',
 'My immediate manager effectively works with people who are different from them to achieve business results',
 'In my team we treat each other fairly, regardless of age, sex, gender identity, family status, race, national origin, nationality, religion, disability or sexual orientation',
 'I feel included in my team',
 'My company takes an interest in my well-being',
 'My work schedule allows me sufficient flexibility to meet my personal/family needs',
 'Career opportunities always go to the most qualified person regardless of age, gender identity, sex, family status, race, national origin, nationality, religion, disability or sexual orientation',
 'Is there anything else you want to share with us?']

##Setting the file

In [17]:
colab_path = '/content/drive/MyDrive/Client Data/'

pathDict = {'K_Survey':'surveywithtarget.csv',
            'K_Survey_Org':'surveywithtargetandorg.csv',
            'Q1_2018':'bianual_survey_1_2018_processed.csv',
            'Q3_2018':'bianual_survey_3_2018_processed.csv',
            'Q1_2019':'bianual_survey_1_2019_processed.csv',
            'Q3_2019':'bianual_survey_3_2019_processed.csv',
            'Q1_2020':'bianual_survey_1_2020_processed.csv'}

#consider training on one quarter and then testing on the next quarter

def setFilePath(filename):
  return pathDict[filename]

In [18]:
import ipywidgets as widgets

w = widgets.Dropdown(
    options=pathDict.keys(),
    description='File Name:',
    disabled=False,
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("File set to %s" % change['new'])
        return change

w.observe(on_change)
display(w)

Dropdown(description='File Name:', options=('K_Survey', 'K_Survey_Org', 'Q1_2018', 'Q3_2018', 'Q1_2019', 'Q3_2…

File set to Q3_2018


In [19]:
FileName = w.value
path = setFilePath(FileName)

In [20]:
surveyFile = colab_path + path
cat = ['object']
num = ['float']
if FileName in 'K_Survey' or FileName in 'K_Survey_Org':
  surveyPd = pd.read_csv(surveyFile,header=None).fillna(0)
  questions = q_cols
  #questions = surveyPd.columns 
  surveyPd.columns = [str(x) for x in list(surveyPd.columns)]
elif 'Q' in FileName:
  surveyPd = pd.read_csv(surveyFile,header=0,index_col=0)
  x1 = surveyPd.select_dtypes(include=cat).fillna('blank').applymap(str)
  x2 = surveyPd.select_dtypes(include=num).fillna(0).applymap(float)
  x3 = surveyPd[surveyPd.columns[-1]]
  surveyPd = pd.concat([x1,x2,x3],axis=1)
  import re 
  #Removing punctuations in string 
  #Using regex 
  questions = [str(y) for y in surveyPd.select_dtypes(include=num).columns] #changing this so the categorical column names are included in the questions var
  surveyPd.columns = [re.sub(r'[^\w\s]', '',s)  for s in list(surveyPd.columns)]
  l1 = [x.replace(" ","") for x in list(surveyPd.select_dtypes(include=cat).columns)]
  l2 = [str(y) for y in list(range(len(surveyPd.select_dtypes(include=num).columns)))]
  l3 = [surveyPd.columns[-1]]
  surveyPd.columns = l1 + l2 + l3
target = surveyPd[surveyPd.columns[-1]]
if target.dtype != bool:
    raise Exception('Last column is not formatted as the Target Boolean Column'.format(target))
    print("File " + FileName + " is formatted incorrectly for python script")
else:
    print("File " + FileName + " is formatted correctly for python script")


File Q3_2018 is formatted correctly for python script


#Reading and cleaning all the dataset for the visuals that require a combination of datasets

In [21]:
#reading and cleaning all the dataset for the visuals that require a combination of datasets

cat = ['object']
num = ['float']

surveyFile0 = colab_path + 'bianual_survey_1_2018_processed.csv'
surveyFile1 = colab_path + 'bianual_survey_3_2018_processed.csv'
surveyFile2 = colab_path + 'bianual_survey_1_2019_processed.csv'
surveyFile3 = colab_path + 'bianual_survey_3_2019_processed.csv'
surveyFile4 = colab_path + 'bianual_survey_1_2020_processed.csv'
surveyFile5 = colab_path + 'surveywithtarget.csv'

Q1_Y2018 = pd.read_csv(surveyFile0, header=0, index_col=0)
x16 = Q1_Y2018.select_dtypes(include=cat).fillna('blank').applymap(str)
x17 = Q1_Y2018.select_dtypes(include=num).fillna(0).applymap(float)
x18 = Q1_Y2018[Q1_Y2018.columns[-1]]
Q1_Y2018 = pd.concat([x16,x17,x18],axis=1)


Q3_Y2018 = pd.read_csv(surveyFile1, header=0, index_col=0)
x1 = Q3_Y2018.select_dtypes(include=cat).fillna('blank').applymap(str)
x2 = Q3_Y2018.select_dtypes(include=num).fillna(0).applymap(float)
x3 = Q3_Y2018[Q3_Y2018.columns[-1]]
Q3_Y2018 = pd.concat([x1,x2,x3],axis=1)

Q1_Y2019 = pd.read_csv(surveyFile2, header=0, index_col=0)
x4 = Q1_Y2019.select_dtypes(include=cat).fillna('blank').applymap(str)
x5 = Q1_Y2019.select_dtypes(include=num).fillna(0).applymap(float)
x6 = Q1_Y2019[Q1_Y2019.columns[-1]]
Q1_Y2019 = pd.concat([x4,x5,x6],axis=1)

Q3_Y2019 = pd.read_csv(surveyFile3, header=0, index_col=0)
x7 = Q3_Y2019.select_dtypes(include=cat).fillna('blank').applymap(str)
x8 = Q3_Y2019.select_dtypes(include=num).fillna(0).applymap(float)
x9 = Q3_Y2019[Q3_Y2019.columns[-1]]
Q3_Y2019 = pd.concat([x7,x8,x9],axis=1)

Q1_Y2020 = pd.read_csv(surveyFile4, header=0, index_col=0)
x10 = Q1_Y2020.select_dtypes(include=cat).fillna('blank').applymap(str)
x11 = Q1_Y2020.select_dtypes(include=num).fillna(0).applymap(float)
x12 = Q1_Y2020[Q1_Y2020.columns[-1]]
Q1_Y2020 = pd.concat([x10,x11,x12],axis=1)

kantar = pd.read_csv(surveyFile5,header=None).fillna(0)
kantar.columns = [str(x) for x in list(kantar.columns)]
kantar.rename(columns={'39': 'target'}, inplace=True)

In [22]:
Q1_Y2018['Source'] = "Quarter 1 2018"
Q3_Y2018['Source'] = "Quarter 3 2018"
Q1_Y2019['Source'] = "Quarter 1 2019"
Q3_Y2019['Source'] = "Quarter 3 2019"
Q1_Y2020['Source'] = "Quarter 1 2020"
kantar['Source'] = "Data Set 1"

In [ ]:
#All the survey data with the target value
agg_df = pd.concat([Q1_Y2018.loc[:,'target':'Source'],Q3_Y2018.loc[:,'target':'Source'],Q1_Y2019.loc[:,'target':'Source'],Q3_Y2019.loc[:,'target':'Source'],Q1_Y2020.loc[:,'target':'Source'],kantar.loc[:,'target':'Source']],axis=0)
agg_df

#Analysis of question: Will you still be here in 12 months?

In [ ]:
#filtering the datasets that have the 12 month question
quest_Q3_2018 = Q3_Y2018[['I intend to stay with the company for at least the next twelve months.', 'target']]
quest_Q1_2019 = Q1_Y2019[['I intend to stay with the company for at least the next twelve months.', 'target']]
quest_Q3_2019 = Q3_Y2019[['I intend to stay with the company for at least the next twelve months.', 'target']]
quest_Q1_2020 = Q1_Y2020[['I intend to stay with the company for at least the next twelve months.', 'target']]


In [26]:
col_names = ['Q3 2018', 'Q1 2019', 'Q3 2019', 'Q1 2020']
cat = ['object']
num = ['float']

combined_df = pd.DataFrame(columns=col_names)
combined_df = pd.concat([quest_Q3_2018,quest_Q1_2019,quest_Q3_2019,quest_Q1_2020], axis=0) #creating a combined df with that 12 month question



x16 = combined_df.select_dtypes(include=cat).fillna('blank').applymap(str)
x17 = combined_df.select_dtypes(include=num).fillna(0).applymap(float)
x18 = combined_df[combined_df.columns[-1]]
combined_df = pd.concat([x16,x17,x18],axis=1)


print(combined_df)

      I intend to stay with the company for at least the next twelve months.  target
0                                                   4.0                        False
1                                                   5.0                        False
2                                                   4.0                        False
3                                                   5.0                        False
4                                                   4.0                        False
...                                                 ...                          ...
5099                                                0.0                        False
5100                                                0.0                        False
5101                                                0.0                        False
5102                                                0.0                        False
5103                                                0.0          

In [27]:
#responses to the 12 month questions across all datasets that had the question

import plotly.express as px

df = combined_df
fig = px.histogram(df, x='I intend to stay with the company for at least the next twelve months.', color='target')
fig.show(renderer='colab')

##Visual for only the people who left the company below

In [28]:

filter = combined_df["target"]==True

df2 = combined_df.where(filter)

fig2 = px.histogram(df2, x='I intend to stay with the company for at least the next twelve months.')
fig2.update_layout(
    title_text='Number of People Who Left vs. Rating Given', # title of plot
    xaxis_title_text='Rating Given To Statement: I intend to stay with the company for at lease the next twelve months.', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig2.show(renderer='colab')

#Visual of ratio of attrition to staying

In [32]:

import plotly.express as px

df3 = surveyPd
fig3 = px.histogram(df3, x='target', title='Number of People Who Stay vs Leave')
fig3.update_layout(
    title_text='Number of People Who Stay vs. Leave', # title of plot
    xaxis_title_text='Stay or Leave', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig3.show(renderer='colab')

#General statistics of the selected survey file

In [31]:
surveyPd.describe().T

,count,mean,std,min,25%,50%,75%,max
0,3550.0,3.765915,1.315799,0.0,4.0,4.0,5.0,5.0
1,3550.0,4.138028,1.292234,0.0,4.0,4.0,5.0,5.0
2,3550.0,3.838310,1.275425,0.0,4.0,4.0,5.0,5.0
3,3550.0,3.838592,1.287006,0.0,4.0,4.0,5.0,5.0
4,3550.0,3.385070,1.360246,0.0,3.0,4.0,4.0,5.0
5,3550.0,3.792958,1.336867,0.0,4.0,4.0,5.0,5.0
6,3550.0,3.861690,1.272981,0.0,4.0,4.0,5.0,5.0
7,3550.0,4.021408,1.309778,0.0,4.0,4.0,5.0,5.0
8,3550.0,4.067042,1.353751,0.0,4.0,4.0,5.0,5.0
9,3550.0,3.947887,1.379453,0.0,4.0,4.0,5.0,5.0


# Generic Pre-Processing Code to Remove blank columns

In [ ]:
#Check if col “c” is a keeper
#(surveyPD[c] > 0).mean()
#[c for c in surveyPD.columns if … ]
#(surveyPD[c] > 0).mean() < threshold

blank = dict(surveyPd.describe().T['mean'] < 0.99)
indexList = [key for key in blank.keys() if blank[key] == True]
surveyPd = surveyPd.drop(indexList, axis=1)

In [ ]:
#surveyPd

# Columns that were dropped because mostly blank

In [ ]:
indexList

# Processed and Cleaned Survey Data

In [ ]:
surveyPd

# Distribution of Organization Attrit Ratio

This comparison is only for the K-survey data

In [ ]:
#The currently only works for K_Survey
import plotly.express as px
cat1 = ['object','bool','float']
df = surveyPd.select_dtypes(include=cat1)
fig = px.histogram(df, x='39', color='41')
fig.show(renderer='colab')

# Exploratory Analysis 

In [ ]:
# creating a list of only numerical values
# Import statements required for Plotly 
import plotly.graph_objs as go
import plotly.tools as tls

#setting up the data
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
newdf = surveyPd.select_dtypes(include=numerics)
numerical = newdf.columns



data = [
    go.Heatmap(
        z= surveyPd[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x= surveyPd[numerical].columns.values,
        y= surveyPd[numerical].columns.values,
        colorscale='Viridis',
        reversescale = False,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of numerical features',
    xaxis = dict(ticks='', nticks=39),
    yaxis = dict(ticks=''),
    width = 900, height = 700,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')
# py.iplot(fig, filename='labelled-heatmap')

In [ ]:
dictQs = {}
listnum = list(range(39))
dictQs = {listnum[i]: q_cols[i] for i in range(len(listnum))}

# Benchmark PCA

In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(surveyPd.select_dtypes(include=numerics))
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Benchmark PCA Results')
fig.show(renderer='colab')

# Testing Zero Shot Classification

In [ ]:
classifiertest = pipeline("zero-shot-classification") 

possible_labels = ["manager", "individual", "satisfaction"] #These labels can be changed as wanted- the order of the label here determines which group it represents - element 0 is Group 1 and etc.


class_results = []
class_results = classifiertest(questions, possible_labels, multi_class=False) #running the zero shot classifier

In [ ]:
#showing the results of the zero shot
class_results

In [ ]:
print(len(class_results))
#initializing the groups
group_1= []
group_2= []
group_3= []
group_4= []

#for loop to determine what group each question belongs to
for c in range(len(class_results)):
  if class_results[c]['labels'][0] == possible_labels[0]:
    good_question = class_results[c]['sequence']
    group_1.append(good_question)
  elif class_results[c]['labels'][0] == possible_labels[1]:
    good_question = class_results[c]['sequence']
    group_2.append(good_question)
  elif class_results[c]['labels'][0] == possible_labels[2]:
    good_question = class_results[c]['sequence']
    group_3.append(good_question)
  elif class_results[c]['labels'][0] == possible_labels[3]:
    good_question = class_results[c]['sequence']
    group_4.append(good_question)

#Printing the results
print("Group 1:   ",group_1)
print("Group 2:   ",group_2)
print("Group 3:   ",group_3)
print("Group 4:   ",group_4)


##Created 4 groupings based on individual satifaction, company satisfaction, team satisfaction , management satifaction

##Group 1 Analysis

In [ ]:
group1_index = dict()
index_list1 =[]

#Making a list and dict of the group 1 questions with their original index values for analysis purposes
for g in group_1:
  for i in range(len(questions)):
    if g == questions[i]:
      group1_index[g]=i
      index_list1.append(i)
      break

print(group1_index)
print(index_list1)



In [ ]:
d = index_list1

#the datafram for the questions in group 1 and the ratings 
G1_DF = pd.DataFrame(columns=d)
G1_DF = surveyPd.select_dtypes(include=numerics).iloc[:,d]

print((G1_DF))


In [ ]:
#correlation matrix of Group 1
data = [
    go.Heatmap(
        z= G1_DF.astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,G1_DF.columns.values)),
        y= list(map(str,G1_DF.columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Group 1',
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

# Happy or Sad Groupings for Group 1

In [ ]:
#Seperating the people into those who are generally happy and those who are generally unhappy based on their question response score 

G1_DF['Average Score'] = G1_DF.iloc[:,0:4].mean(axis=1)
happyDF = G1_DF[G1_DF['Average Score'] > 2.50]
sadDF =  G1_DF[G1_DF['Average Score'] <= 2.50]

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 1 Happy PCA Results')
fig.show(renderer='colab')

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 1 Sad PCA Results')
fig.show(renderer='colab')

# Group 2 Analysis

In [ ]:
group2_index = dict()
index_list2 =[]

#Making a list and dict of the group 1 questions with their original index values for analysis purposes
for g in group_2:
  for i in range(len(questions)):
    if g == questions[i]:
      group2_index[g]=i
      index_list2.append(i)
      break

print(group2_index)
print(index_list2)

In [ ]:
#Defining the DF for group 2's questions
e = index_list2

G2_DF = pd.DataFrame(columns=e)
G2_DF = surveyPd.select_dtypes(include=numerics).iloc[:,e]

print((G2_DF))

In [ ]:

data = [
    go.Heatmap(
        z= G2_DF.astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,G2_DF.columns.values)),
        y= list(map(str,G2_Df.columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Group 2',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

# Happy or Sad Groupings for Group 2

In [ ]:
G2_DF['Average Score'] = G2_DF.iloc[:,0:4].mean(axis=1)
happyDF = G2_DF[G2_DF['Average Score'] > 2.50]
sadDF =  G2_DF[G2_DF['Average Score'] <= 2.50]

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 2 Happy PCA Results')
fig.show(renderer='colab')

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 2 Sad PCA Results')
fig.show(renderer='colab')

# Group 3 Analysis

In [16]:
group3_index = dict()
index_list3 =[]

#Making a list and dict of the group 1 questions with their original index values for analysis purposes
for g in group_3:
  for i in range(len(questions)):
    if g == questions[i]:
      group3_index[g]=i
      index_list3.append(i)
      break

print(group3_index)
print(index_list3)

NameError: ignored

In [ ]:
f = index_list3
G3_DF = pd.DataFrame(columns=f)
G3_DF = surveyPd.select_dtypes(include=numerics).iloc[:,f]


In [ ]:
print(G3_DF)

In [ ]:

data = [
    go.Heatmap(
        z= G3_DF.astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,G3_DF.columns.values)),
        y= list(map(str,G3_Df.columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Group 3',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

# Happy or Sad Groupings for Group 3

In [ ]:
G3_DF['Average Score'] = G3_DF.iloc[:,0:4].mean(axis=1)
happyDF = G3_DF[G3_DF['Average Score'] > 2.50]
sadDF =  G3_DF[G3_DF['Average Score'] <= 2.50]

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 3 Happy PCA Results')
fig.show(renderer='colab')

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 3 Sad PCA Results')
fig.show(renderer='colab')

# Group 4 Analysis

In [ ]:
group4_index = dict()
index_list4 =[]

#Making a list and dict of the group 1 questions with their original index values for analysis purposes
for g in group_4:
  for i in range(len(questions)):
    if g == questions[i]:
      group4_index[g]=i
      index_list4.append(i)
      break

print(group4_index)
print(index_list4)

In [ ]:
g = index_list4
G4_DF = pd.DataFrame(columns=g)
G4_DF = surveyPd.select_dtypes(include=numerics).iloc[:,g]

G4_DF

In [ ]:

data = [
    go.Heatmap(
        z= G4_DF.astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,G4_DF.columns.values)),
        y= list(map(str,G4_Df.columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Group 4',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

# Happy or Sad for Group 4

In [ ]:
G4_DF['Average Score'] = G4_DF.iloc[:,0:4].mean(axis=1)
happyDF = G4_DF[G4_DF['Average Score'] > 2.50]
sadDF =  G4_DF[G4_DF['Average Score'] <= 2.50]

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 4 Happy PCA Results')
fig.show(renderer='colab')

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Group 4 Sad PCA Results')
fig.show(renderer='colab')

## The following groupings were done manually by a memeber of the team. These only apply to the questions in the K-survey dataset

# Individual Employee Questions

---



In [ ]:
emplSatis = [0,20,28,36]
for key, value in dictQs.items():
  if key in emplSatis:
    print (key, value)

d = emplSatis
indDf = pd.DataFrame(columns=d)
indDf = surveyPd.loc[:,emplSatis]



In [ ]:
numerical = emplSatis
data = [
    go.Heatmap(
        z= indDf[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,indDf[numerical].columns.values)),
        y= list(map(str,indDf[numerical].columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Individual Questions',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

In [ ]:
indDf['Average Score'] = indDf.iloc[:,0:4].mean(axis=1)
happyDF = indDf[indDf['Average Score'] > 2.50]
sadDF =  indDf[indDf['Average Score'] <= 2.50]

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Individual Happy PCA Results')
fig.show(renderer='colab')

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:4])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Individual Sad PCA Results')
fig.show(renderer='colab')

# Company Questions

In [ ]:
comSatis = [1,5,6,18,19,24,25,26,29,30,35,37]
for key, value in dictQs.items():
  if key in comSatis:
    print (key, value)
d = comSatis
comDf = pd.DataFrame(columns=d)
comDf = surveyPd.loc[:,comSatis]

numerical = comSatis
data = [
    go.Heatmap(
        z= comDf[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,comDf[numerical].columns.values)),
        y= list(map(str,comDf[numerical].columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Company Questions',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show(renderer='colab')

In [ ]:
comDf['Average Score'] = comDf.iloc[:,0:11].mean(axis=1)
happyDF = comDf[comDf['Average Score'] > 2.50]
sadDF =  comDf[comDf['Average Score'] <= 2.50]
happyDF

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:12])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Company Happy PCA Results')
fig.show()

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:12])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Company Sad PCA Results')
fig.show()

# Team Related Questions


In [ ]:
teamSatis = [2,4,10,11,21,22,23,27,33,34]
for key, value in dictQs.items():
  if key in teamSatis:
    print (key, value)
d = teamSatis
teamDf = pd.DataFrame(columns=d)
teamDf = surveyPd.loc[:,teamSatis]


numerical = teamSatis
data = [
    go.Heatmap(
        z= teamDf[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,teamDf[numerical].columns.values)),
        y= list(map(str,teamDf[numerical].columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Team Questions',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
teamDf['Average Score'] = teamDf.iloc[:,0:10].mean(axis=1)
happyDF = teamDf[teamDf['Average Score'] > 2.50]
sadDF =  teamDf[teamDf['Average Score'] <= 2.50]
happyDF

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF.iloc[:,0:10])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Team Happy PCA Results')
fig.show()

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF.iloc[:,0:10])
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Team SAD PCA Results')
fig.show()

# Management Related Questions

In [ ]:
manSatis = [13,14,15,16,17,31,32]
for key, value in dictQs.items():
  if key in manSatis:
    print (key, value)
d = manSatis
manDf = pd.DataFrame(columns=d)
manDf = surveyPd.loc[:,manSatis]

numerical = manSatis
data = [
    go.Heatmap(
        z= manDf[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x= list(map(str,manDf[numerical].columns.values)),
        y= list(map(str,manDf[numerical].columns.values)),
        colorscale='portland',
        reversescale = True,
#         text = True ,
        opacity = 1.0
        
    )
]
layout = go.Layout(
    title='Pearson Correlation of Management Questions',
    # xaxis = list(map(str, numerical)) ,
    # yaxis = list(map(str, numerical)),
    width = 500, height = 500,
    
)
layout

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
manDf['Average Score'] = manDf.iloc[:,0:6].mean(axis=1)
happyDF = manDf[manDf['Average Score'] > 2.50]
sadDF =  manDf[manDf['Average Score'] <= 2.50]
manDf

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(happyDF)
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Management Happy PCA Results')
fig.show()

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(sadDF)
principalDf = pd.DataFrame(data=principalComponents,columns = ['principal component 1', 'principal component 2'])
fig=px.scatter(x=principalDf['principal component 1'], y =principalDf['principal component 2'],title='Management Sad PCA Results')
fig.show()

# Average the groupings and merge into one dataframe

In [ ]:
d = ['employeeSatisfaction','companySatisfaction','teamSatisfaction','managementSatisfaction','Attrit']
groupeddf = pd.DataFrame(columns=d)
groupeddf['employeeSatisfaction'] = surveyPd.loc[:,emplSatis].mean(axis=1).map(int)
groupeddf['companySatisfaction'] = surveyPd.loc[:,comSatis].mean(axis=1).map(int)
groupeddf['teamSatisfaction'] = surveyPd.loc[:,teamSatis].mean(axis=1).map(int)
groupeddf['managementSatisfaction'] = surveyPd.loc[:,manSatis].mean(axis=1).map(int)
groupeddf['Attrit'] = surveyPd.loc[:,39]
groupeddf